In [6]:
import pandas
import numpy

features = ['accommodates', 'bedrooms', 'bathrooms', 'beds', 'price', 'minimum_nights', 'maximum_nights',
            'number_of_reviews']
dc_listings = pandas.read_csv("E:\\Workspace\\jupyter_notebook\\notebook_idata_lesson01\\KNN\\listings.csv")
dc_listings = dc_listings[features]
print(dc_listings.shape)

dc_listings[:5]


(3723, 8)


,accommodates,bedrooms,bathrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews
0,4,1.0,1.0,2.0,$160.00,1,1125,0
1,6,3.0,3.0,3.0,$350.00,2,30,65
2,1,1.0,2.0,1.0,$50.00,2,1125,1
3,2,1.0,1.0,1.0,$95.00,1,1125,0
4,4,1.0,1.0,1.0,$50.00,7,1125,0


In [9]:
# 假设我们的房子有3个房间
our_acc_value = 3
dc_listings["distance"] = numpy.abs(dc_listings.accommodates - our_acc_value)
# 查看一共有多少种距离
dc_listings.distance.value_counts().sort_index()


0      461
1     2294
2      503
3      279
4       35
5       73
6       17
7       22
8        7
9       12
10       2
11       4
12       6
13       8
Name: distance, dtype: int64

In [10]:
# 打乱顺序
dc_listings = dc_listings.sample(frac=1, random_state=0)
dc_listings = dc_listings.sort_values("distance")
dc_listings.price.head()

740     $165.00
453     $207.00
602     $130.00
2348    $138.00
3119    $125.00
Name: price, dtype: object

In [20]:
# 转换price里面的数据格式，去掉$符转换成数值型
dc_listings["price"] = dc_listings["price"].replace("\$|,", "").astype(float)

# 计算前5个的平均值
mean_price = dc_listings["price"].iloc[:5].mean()
print(mean_price)


153.0


In [21]:
# 制定训练集和测试集
dc_listings.drop("distance", axis=1)

# 75%是训练集，25%是测试集
train_df = dc_listings.copy().iloc[:2792]
test_df = dc_listings.copy().iloc[2792:]


In [31]:
# 基于单变量预测价格
def predict_price(new_listing_value, feature_column):
    temp_df = train_df
    temp_df["distance"] = numpy.abs(dc_listings[feature_column] - new_listing_value)
    temp_df = temp_df.sort_values("distance")
    knn_5 = temp_df.price.iloc[:5]
    predicated_price = knn_5.mean()
    return predicated_price


# 调用predict_price预测
test_df["predicted_price"] = test_df.accommodates.apply(predict_price, feature_column="accommodates")


In [32]:
# 计算RMSE
test_df["squared_error"] = (test_df["predicted_price"] - test_df["price"]) ** 2
mse = test_df["squared_error"].mean()
rmse = mse ** (1 / 2)
print(rmse)


229.03265260947379


In [34]:
# 不同变量各自计算RMSE
for feature in ["accommodates", "bedrooms", "bathrooms", "number_of_reviews"]:
    test_df["predicted_price"] = test_df[feature].apply(predict_price, feature_column=feature)
    test_df["squared_error"] = (test_df["predicted_price"] - test_df["price"]) ** 2
    mse = test_df["squared_error"].mean()
    rmse = mse ** (1 / 2)
    print("RMSE for the {} column: {}".format(feature, rmse))


RMSE for the accommodates column: 229.03265260947379


RMSE for the bedrooms column: 214.33995002869366


RMSE for the bathrooms column: 235.21466649435237


RMSE for the number_of_reviews column: 249.9682819836384
